In [62]:
import re
import time
import pandas as pd

In [63]:
# Function to convert duration string to decimal hours
def duration_to_hours(duration_str):
    if isinstance(duration_str, str):
        hours = 0
        minutes = 0
        hr_match = re.search(r'(\d+)\s*hr', duration_str)
        min_match = re.search(r'(\d+)\s*min', duration_str)
        if hr_match:
            hours = int(hr_match.group(1))
        if min_match:
            minutes = int(min_match.group(1))
        total_hours = hours + minutes / 60
        return round(total_hours, 2)
    else:
        return None
    
def duration_to_minutes(duration_str):
    if isinstance(duration_str, str):
        hours = 0
        minutes = 0
        hr_match = re.search(r'(\d+)\s*hr', duration_str)
        min_match = re.search(r'(\d+)\s*min', duration_str)
        if hr_match:
            hours = int(hr_match.group(1))
        if min_match:
            minutes = int(min_match.group(1))
        return hours * 60 + minutes
    else:
        return None

In [64]:
# Load the CSV file into a DataFrame
csv_file_path = 'results.csv'
airport_file_path = 'settings/airports.csv'
df = pd.read_csv(csv_file_path)
df_airports = pd.read_csv(airport_file_path)

# Clean 'price' column: remove commas, convert to numeric, turn errors into NaN
df['price'] = df['price'].replace(',', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Drop rows where price is NaN
df = df.dropna(subset=['price'])

# Now safely convert to float
df['price'] = df['price'].astype(float)

In [65]:
# Sort the DataFrame by price (ascending)
df_sorted = df.sort_values(by='price')

# Show the cheapest 10 options
df_sorted.head(10)


,timestamp,from,to,outbound,inbound,price,airline,time,duration,type
2642,1.747326e+09,NAP,RHO,2025-08-09,2025-08-24,276.0,"Lufthansa, Discover Airlines",6:50PM 8:50AM+1,13 hr,1 stop
2643,1.747326e+09,NAP,RHO,2025-08-09,2025-08-24,300.0,7 hr 45 min FRA,7:50PM 8:15AM+1,11 hr 25 min,1 stop
2645,1.747326e+09,NAP,RHO,2025-08-09,2025-08-24,329.0,7 hr 20 min VIE,8:00PM 7:35PM+1,22 hr 35 min,1 stop
2646,1.747326e+09,NAP,RHO,2025-08-09,2025-08-24,358.0,Lufthansa,8:00PM 7:20PM+1,22 hr 20 min,2 stops
159,1.747325e+09,FCO,RHO,2025-08-03,2025-08-18,371.0,ITA,10:20PM 5:20AM+1,6 hr,1 stop
156,1.747325e+09,FCO,RHO,2025-08-03,2025-08-18,371.0,Aegean,6:25PM 12:10AM+1,4 hr 45 min,1 stop
195,1.747325e+09,FCO,RHO,2025-08-10,2025-08-25,378.0,ITA,10:20PM 5:20AM+1,6 hr,1 stop
2625,1.747326e+09,NAP,RHO,2025-08-06,2025-08-21,397.0,easyJet,6:25AM 9:35AM,2 hr 10 min,Nonstop
3881,1.747327e+09,TRN,RHO,2025-08-05,2025-08-20,403.0,"Air Dolomiti, Discover AirlinesLufthansa",5:25PM 9:35AM+1,15 hr 10 min,1 stop
192,1.747325e+09,FCO,RHO,2025-08-10,2025-08-25,404.0,Aegean,6:25PM 12:10AM+1,4 hr 45 min,1 stop


In [66]:
# Create helper column for sorting
df['duration_hours'] = df['duration'].apply(duration_to_hours)

# Sort the DataFrame by duration (ascending)
df_sorted_by_duration = df.sort_values(by='duration_hours')

# Display only desired columns — keeping original 'duration'
df_sorted_by_duration[['from', 'to', 'outbound', 'inbound', 'price', 'duration', 'type', 'airline']].head(10)


,from,to,outbound,inbound,price,duration,type,airline
2625,NAP,RHO,2025-08-06,2025-08-21,397.0,2 hr 10 min,Nonstop,easyJet
2624,NAP,RHO,2025-08-05,2025-08-20,408.0,2 hr 10 min,Nonstop,easyJet
194,FCO,RHO,2025-08-10,2025-08-25,600.0,2 hr 20 min,Nonstop,ITA
157,FCO,RHO,2025-08-03,2025-08-18,495.0,2 hr 20 min,Nonstop,1 hr 50 min ATH
158,FCO,RHO,2025-08-03,2025-08-18,495.0,2 hr 25 min,Nonstop,ITA
193,FCO,RHO,2025-08-10,2025-08-25,525.0,2 hr 25 min,Nonstop,1 hr 50 min ATH
2620,NAP,RHO,2025-08-03,2025-08-18,559.0,4 hr 25 min,1 stop,"SWISS, Edelweiss AirOperated by Helvetic"
2622,NAP,RHO,2025-08-03,2025-08-18,644.0,4 hr 30 min,1 stop,ITA
156,FCO,RHO,2025-08-03,2025-08-18,371.0,4 hr 45 min,1 stop,Aegean
192,FCO,RHO,2025-08-10,2025-08-25,404.0,4 hr 45 min,1 stop,Aegean


In [68]:
# Option for more detailed airport information
def get_airport_info(code):
    if code in airport_name:
        return f"{code} ({airport_name.get(code)}, {airport_city.get(code)})"
    return code
# Apply the conversion
df['duration_minutes'] = df['duration'].apply(duration_to_minutes)

# Create a ranking based on both price and duration
# Normalize both metrics to a 0-1 scale to make them comparable
df['price_norm'] = (df['price'] - df['price'].min()) / (df['price'].max() - df['price'].min())
df['duration_norm'] = (df['duration_minutes'] - df['duration_minutes'].min()) / (df['duration_minutes'].max() - df['duration_minutes'].min())

# Create a composite score - equal weight for price and duration
# Lower score is better (cheaper and shorter)
df['score'] = 0.5 * df['price_norm'] + 0.5 * df['duration_norm']

# Sort by the composite score
best_flights = df.sort_values('score')

# Create dictionaries for quick lookup of airport cities and countries
airport_city = dict(zip(df_airports['IATA'], df_airports['City']))
airport_country = dict(zip(df_airports['IATA'], df_airports['Country']))

# Add city and country columns to the DataFrame
best_flights['from_city'] = best_flights['from'].map(airport_city)
best_flights['from_country'] = best_flights['from'].map(airport_country)
best_flights['to_city'] = best_flights['to'].map(airport_city)
best_flights['to_country'] = best_flights['to'].map(airport_country)

# Display the top flights with city and country information
print("\nBest flights (balancing price and duration):")
for index, row in best_flights.head().iterrows():
    from_code = row['from']
    to_code = row['to']
    
    # Get cities and countries with fallback to code if not found
    from_city = row['from_city'] if pd.notna(row['from_city']) else from_code
    from_country = row['from_country'] if pd.notna(row['from_country']) else ""
    to_city = row['to_city'] if pd.notna(row['to_city']) else to_code
    to_country = row['to_country'] if pd.notna(row['to_country']) else ""
    
    print(f"{from_code} ({from_city}, {from_country}) → {to_code} ({to_city}, {to_country}): {row['price']}€, {row['duration']}, {row['airline']}, {row['type']}")

# Drop calculation columns while keeping the new city/country columns
columns_to_drop = ["timestamp", "duration_minutes", "price_norm", "duration_norm", "score"]
best_flights_clean = best_flights.drop(columns=columns_to_drop)

# Reorder columns to put the city and country information next to the airport codes
column_order = ['from', 'from_city', 'from_country', 'to', 'to_city', 'to_country', 
                'outbound', 'inbound', 'price', 'airline', 'time', 'duration', 'type']
best_flights_clean = best_flights_clean[column_order]

# Save the best flights to a CSV file
filename = f'{int(time.time())}_best_flights.csv'
best_flights_clean.to_csv(filename, index=False)
print(f"\nSaved detailed flight information to {filename}")


Best flights (balancing price and duration):
NAP (Napoli, Italy) → RHO (Rhodes, Greece): 397.0€, 2 hr 10 min, easyJet, Nonstop
NAP (Napoli, Italy) → RHO (Rhodes, Greece): 408.0€, 2 hr 10 min, easyJet, Nonstop
FCO (Rome, Italy) → RHO (Rhodes, Greece): 495.0€, 2 hr 20 min, 1 hr 50 min ATH, Nonstop
FCO (Rome, Italy) → RHO (Rhodes, Greece): 495.0€, 2 hr 25 min, ITA, Nonstop
FCO (Rome, Italy) → RHO (Rhodes, Greece): 525.0€, 2 hr 25 min, 1 hr 50 min ATH, Nonstop

Saved detailed flight information to 1747335987_best_flights.csv


In [84]:
# Define the destination codes you're interested in
target_destinations = ["BOG", "MDE", "CLO", "CTG", "BAQ", "CUN", "SJU", "GIG", "GRU", "SDU"]

# Filter flights to only include those with destinations in your target list
filtered_flights = df[df['to'].isin(target_destinations)].copy()

# Add city and country information to filtered_flights
filtered_flights['to_city'] = filtered_flights['to'].map(airport_city)
filtered_flights['to_country'] = filtered_flights['to'].map(airport_country)
filtered_flights['from_city'] = filtered_flights['from'].map(airport_city)
filtered_flights['from_country'] = filtered_flights['from'].map(airport_country)

# Create normalized metrics for price and duration
filtered_flights['price_norm'] = (filtered_flights['price'] - filtered_flights['price'].min()) / (filtered_flights['price'].max() - filtered_flights['price'].min())
filtered_flights['duration_norm'] = (filtered_flights['duration_minutes'] - filtered_flights['duration_minutes'].min()) / (filtered_flights['duration_minutes'].max() - filtered_flights['duration_minutes'].min())

# Create a composite score (equal weight for price and duration)
filtered_flights['score'] = 0.5 * filtered_flights['price_norm'] + 0.5 * filtered_flights['duration_norm']

# Sort by the composite score
best_flights = filtered_flights.sort_values('score')

# Create dictionaries for quick lookup
airport_city = dict(zip(df_airports['IATA'], df_airports['City']))
airport_country = dict(zip(df_airports['IATA'], df_airports['Country']))

# Add city and country information
best_flights['to_city'] = best_flights['to'].map(airport_city)
best_flights['to_country'] = best_flights['to'].map(airport_country)
best_flights['from_city'] = best_flights['from'].map(airport_city)
best_flights['from_country'] = best_flights['from'].map(airport_country)

# Group by destination and show the best flight for each
print("Best flight for each destination in South America:")
for destination in target_destinations:
    dest_flights = best_flights[best_flights['to'] == destination]
    if not dest_flights.empty:
        best_flight = dest_flights.iloc[0]
        from_code = best_flight['from']
        to_code = best_flight['to']
        
        # Get city and country info with fallbacks
        from_city = best_flight['from_city'] if pd.notna(best_flight['from_city']) else from_code
        from_country = best_flight['from_country'] if pd.notna(best_flight['from_country']) else ""
        to_city = best_flight['to_city'] if pd.notna(best_flight['to_city']) else to_code
        to_country = best_flight['to_country'] if pd.notna(best_flight['to_country']) else ""

        print(f"{from_code} ({from_city}, {from_country}) → {to_code} ({to_city}, {to_country}) - {best_flight['price']}€: from {best_flight['outbound']} to {best_flight['inbound']}, {best_flight['duration']}, {best_flight['airline']}, {best_flight['type']}")
    else:
        print(f"No flights found to {destination}")

# Calculate price statistics for each destination
print("\nPrice statistics by destination:")
for destination in target_destinations:
    dest_flights = filtered_flights[filtered_flights['to'] == destination]
    if not dest_flights.empty:
        to_city = dest_flights.iloc[0]['to_city'] if pd.notna(dest_flights.iloc[0]['to_city']) else destination
        
        # Calculate price statistics
        min_price = dest_flights['price'].min()
        max_price = dest_flights['price'].max()
        avg_price = dest_flights['price'].mean()
        median_price = dest_flights['price'].median()
        
        print(f"\n{destination} ({to_city}):")
        print(f"  Minimum price: {min_price}€")
        print(f"  Maximum price: {max_price}€")
        print(f"  Average price: {avg_price:.2f}€")
        print(f"  Median price: {median_price}€")
        print(f"  Number of flights: {len(dest_flights)}")
    else:
        print(f"\n{destination}: No flights found")

Best flight for each destination in South America:
FCO (Rome, Italy) → BOG (Bogota, Colombia) - 1191.0€: from 2025-08-03 to 2025-08-18, 14 hr 10 min, KLM, 1 stop
FCO (Rome, Italy) → MDE (Medellín, Colombia) - 1487.0€: from 2025-08-02 to 2025-08-17, 15 hr 5 min, Air Europa, 1 stop
TRN (Caselle Torinese (TO), Italy) → CLO (Cali, Colombia) - 2080.0€: from 2025-08-05 to 2025-08-20, 16 hr 58 min, KLM, AviancaOperated by KLM Cityhopper, 2 stops
FCO (Rome, Italy) → CTG (Cartagena, Colombia) - 1972.0€: from 2025-08-09 to 2025-08-24, 17 hr 15 min, Air Canada, Avianca, 2 stops
FCO (Rome, Italy) → BAQ (Barranquilla, Colombia) - 2098.0€: from 2025-08-05 to 2025-08-20, 17 hr 44 min, American, 2 stops
FCO (Rome, Italy) → CUN (Cancún, Mexico) - 1313.0€: from 2025-08-09 to 2025-08-24, 14 hr 20 min, 3 hr 40 min FRA, 1 stop
NAP (Napoli, Italy) → SJU (San Juan, Puerto Rico) - 899.0€: from 2025-08-05 to 2025-08-20, 15 hr 57 min, United, 1 stop
FCO (Rome, Italy) → GIG (Rio De Janeiro, Brazil) - 1686.0€: fr